In [36]:
import pandas as pd
from api import M2M

m2m = M2M('sobczynski','Fallow4Sure!')

first URL: https://m2m.cr.usgs.gov/api/api/json/stable


In [37]:
############################
## GET LOCATION OF CENTER ##
############################
wd = "C:/Users/asobc/PycharmProjects/fallow/"
wrs_ref = pd.read_excel(f'{wd}WRScornerPoints_0.xlsx')
path = 170
row_name = 64
path_row = "170064"
row_index = wrs_ref[(wrs_ref['PATH'] == path) & (wrs_ref['ROW'] == row_name)].index[0]

center_lat = wrs_ref.at[row_index, 'CTR LAT']
center_lon = wrs_ref.at[row_index, 'CTR LON']
print(center_lon)
print(center_lat)

33.053446
-5.785213


In [38]:
###########################
##### COMPILE ALL IDS #####
###########################

# Get rid of Tier 2 and scenes and scenes with lots of cloud cover
scene_list = []
first_year = 1984
last_year = 2023
planting_start_month = '02'
planting_start_day = '01'
planting_end_month = '03'
planting_end_day = '31'
growing_start_month = '04'
growing_start_day = '01'
growing_end_month = '06'
growing_end_day = '30'
target_grow_month = "06"
target_grow_day = "15"
target_plant_month = "02"
target_plant_day = "15"

for year in range(first_year, 2013):
    params = {
        "datasetName": 'landsat_tm_c2_l2',
        "startDate": f"{year}-{planting_start_month}-{planting_start_day}",
        "endDate": f"{year}-{growing_end_month}-{growing_end_day}",
        "geoJsonType": "Polygon",
        "geoJsonCoords": [[[center_lon, center_lat],
                           [center_lon, (center_lat - .25)],
                           [(center_lon + .25), (center_lat - .25)],
                           [(center_lon + .25), center_lat],
                           [center_lon, center_lat]]],
        "maxCC": 70,
        "maxResults": 10000
    }
    scenes = m2m.searchScenes(**params)
    for result in scenes['results']:
        entityId = result['displayId']
        scene_list.append(entityId)

for year in range(1999, last_year + 1):
    params = {
        "datasetName": 'landsat_etm_c2_l2',
        "startDate": f"{year}-{planting_start_month}-{planting_start_day}",
        "endDate": f"{year}-{growing_end_month}-{growing_end_day}",
        "geoJsonType": "Polygon",
        "geoJsonCoords": [[[center_lon, center_lat],
                           [center_lon, (center_lat - .25)],
                           [(center_lon + .25), (center_lat - .25)],
                           [(center_lon + .25), center_lat],
                           [center_lon, center_lat]]],
        "maxCC": 70,
        "maxResults": 10000
    }
    scenes = m2m.searchScenes(**params)
    for result in scenes['results']:
        entityId = result['displayId']
        scene_list.append(entityId)

for year in range(2013, last_year + 1):
    params = {
        "datasetName": 'landsat_ot_c2_l2',
        "startDate": f"{year}-{planting_start_month}-{planting_start_day}",
        "endDate": f"{year}-{growing_end_month}-{growing_end_day}",
        "geoJsonType": "Polygon",
        "geoJsonCoords": [[[center_lon, center_lat],
                           [center_lon, (center_lat - .25)],
                           [(center_lon + .25), (center_lat - .25)],
                           [(center_lon + .25), center_lat],
                           [center_lon, center_lat]]],
        "maxCC": 70,
        "maxResults": 10000
    }
    scenes = m2m.searchScenes(**params)
    for result in scenes['results']:
        entityId = result['displayId']
        scene_list.append(entityId)

scene_list[:] = [scene for scene in scene_list if not scene.endswith('T2') and path_row in scene]

#print(scene_list)

['LT05_L2SP_170064_19840606_20200918_02_T1', 'LT05_L2SP_170064_19840521_20200918_02_T1', 'LT05_L2SP_170064_19850305_20200918_02_T1', 'LT04_L2SP_170064_19880218_20200917_02_T1', 'LT05_L2SP_170064_19890604_20200916_02_T1', 'LT05_L2SP_170064_19900623_20200915_02_T1', 'LT04_L2SP_170064_19900207_20200916_02_T1', 'LT05_L2SP_170064_19910423_20200915_02_T1', 'LT05_L2SP_170064_19910407_20200915_02_T1', 'LT05_L2SP_170064_19920628_20200914_02_T1', 'LT05_L2SP_170064_19930412_20200914_02_T1', 'LT05_L2SP_170064_19930327_20200914_02_T1', 'LT05_L2SP_170064_19940618_20200913_02_T1', 'LT05_L2SP_170064_19950402_20200912_02_T1', 'LT05_L2SP_170064_19950317_20200912_02_T1', 'LT05_L2SP_170064_19950213_20200912_02_T1', 'LT05_L2SP_170064_19960216_20200912_02_T1', 'LT05_L2SP_170064_19970626_20200910_02_T1', 'LT05_L2SP_170064_19970610_20200910_02_T1', 'LT05_L2SP_170064_19980629_20200909_02_T1', 'LT05_L2SP_170064_19980410_20200909_02_T1', 'LT05_L2SP_170064_19990429_20200908_02_T1', 'LT05_L2SP_170064_19990224_2020

In [40]:
#################################
#### DATAFRAME OF SCENE INFO ####
#################################

from datetime import datetime, timedelta

# make a dataframe that is id, year, month, day, distance from central date
scene_df = pd.DataFrame({'scene_id': scene_list})

scene_df['year'] = ""
scene_df['month'] = ""
scene_df['day'] = ""
scene_df['season'] = ""
scene_df['target_date'] = ""

for index, row in scene_df.iterrows():
    scene_id = row['scene_id']
    year = scene_id[17:21]
    month = scene_id[21:23]
    day = scene_id[23:25]
    scene_df.at[index,'year'] = year
    scene_df.at[index,'month'] = month
    scene_df.at[index,'day'] = day

scene_df['date'] = pd.to_datetime(scene_df[['day', 'month', 'year']], format='%Y-%m-%d').dt.date
plant_end = pd.to_datetime(scene_df['year'].astype(str) + f'-{planting_end_month}-{planting_end_day}', format='%Y-%m-%d').dt.date
plant_target = pd.to_datetime(scene_df['year'].astype(str) + f'-{target_plant_month}-{target_plant_day}', format='%Y-%m-%d').dt.date
grow_target = pd.to_datetime(scene_df['year'].astype(str) + f'-{target_grow_month}-{target_grow_day}', format='%Y-%m-%d').dt.date

scene_df.loc[scene_df['date'] <= plant_end, 'season'] = 'sow'
scene_df.loc[scene_df['date'] <= plant_end, 'target_date'] = plant_target
scene_df.loc[scene_df['date'] > plant_end, 'season'] = 'gro'
scene_df.loc[scene_df['date'] > plant_end, 'target_date'] = grow_target

scene_df['target_date'] = pd.to_datetime(scene_df['target_date'])
scene_df['date'] = pd.to_datetime(scene_df['date'])

scene_df['days_from_target'] = (scene_df['target_date'] - scene_df['date']).dt.days.abs()
scene_df.drop(columns=['target_date'], inplace=True)

print(scene_df)

                                     scene_id  year month day season  \
0    LT05_L2SP_170064_19840606_20200918_02_T1  1984    06  06    gro   
1    LT05_L2SP_170064_19840521_20200918_02_T1  1984    05  21    gro   
2    LT05_L2SP_170064_19850305_20200918_02_T1  1985    03  05    sow   
3    LT04_L2SP_170064_19880218_20200917_02_T1  1988    02  18    sow   
4    LT05_L2SP_170064_19890604_20200916_02_T1  1989    06  04    gro   
..                                        ...   ...   ...  ..    ...   
246  LC09_L2SP_170064_20230322_20230324_02_T1  2023    03  22    sow   
247  LC09_L2SP_170064_20230306_20230308_02_T1  2023    03  06    sow   
248  LC08_L2SP_170064_20230226_20230301_02_T1  2023    02  26    sow   
249  LC09_L2SP_170064_20230218_20230310_02_T1  2023    02  18    sow   
250  LC09_L2SP_170064_20230202_20230311_02_T1  2023    02  02    sow   

          date  days_from_target  
0   1984-06-06                 9  
1   1984-05-21                25  
2   1985-03-05                

In [44]:
#################################
# TRIM TO THREE RELEVANT SCENES #
#################################

scene_df['year_season'] = scene_df['year'].astype(str) + '_' + scene_df['season']
grouped = scene_df.groupby(scene_df['year_season'])
#grouped = scene_df.groupby(['year', 'season']).size().unstack(fill_value=0)
def filter_func(x):
    if len(x) > 3:
        sorted_group = x.sort_values(by='days_from_target', ascending=True)
        return sorted_group.iloc[:3]
    else:
        return x

three_df = grouped.apply(filter_func)
three_df = three_df.reset_index(drop=True)
print(three_df)

                                     scene_id  year month day season  \
0    LT05_L2SP_170064_19840606_20200918_02_T1  1984    06  06    gro   
1    LT05_L2SP_170064_19840521_20200918_02_T1  1984    05  21    gro   
2    LT05_L2SP_170064_19850305_20200918_02_T1  1985    03  05    sow   
3    LT04_L2SP_170064_19880218_20200917_02_T1  1988    02  18    sow   
4    LT05_L2SP_170064_19890604_20200916_02_T1  1989    06  04    gro   
..                                        ...   ...   ...  ..    ...   
137  LC09_L2SP_170064_20230610_20230612_02_T1  2023    06  10    gro   
138  LC09_L2SP_170064_20230626_20230628_02_T1  2023    06  26    gro   
139  LC09_L2SP_170064_20230218_20230310_02_T1  2023    02  18    sow   
140  LC08_L2SP_170064_20230226_20230301_02_T1  2023    02  26    sow   
141  LC09_L2SP_170064_20230202_20230311_02_T1  2023    02  02    sow   

          date  days_from_target year_season  
0   1984-06-06                 9    1984_gro  
1   1984-05-21                25    1984_

C:\Users\asobc\AppData\Local\Temp\ipykernel_30204\1933415247.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  three_df = grouped.apply(filter_func)


In [88]:
################################
###### ENSURE SOW AND GRO ######
################################

group_both = scene_df.groupby(['year', 'season']).size().unstack(fill_value=0)
good_years = group_both[(group_both['sow'] > 0) & (group_both['gro'] > 0)]
covered = good_years.index.tolist()

three_df_2 = three_df[three_df['year'].isin(covered)]
print(len(three_df_2['scene_id']))

120


In [47]:
#################################
#### TEXT FILE FOR REFERENCE ####
#################################

import os

def save_list_to_txt(lst, filename):
    with open(filename, 'w') as file:
        for item in lst:
            file.write(str(item) + '\n')

# Save the list to a .txt file
save_list_to_txt(three_df_2['scene_id'].tolist(), os.path.join(wd,f'{path}_{row_name}_scenes.txt'))

In [60]:
###############################
##### INITIALIZE ESPA API #####
###############################
# from here: https://espa.cr.usgs.gov/static/docs/examples/api_demo.py
import platform
import requests
import json
import getpass

host = 'https://espa.cr.usgs.gov/api/v1/'
username = 'sobczynski'
password = 'Fallow4Sure!'

In [61]:
##############################
#### FUNCTION TO INTERACT ####
##############################

def espa_api(endpoint, verb='get', body=None, uauth=None):
    """ Suggested simple way to interact with the ESPA JSON REST API """
    auth_tup = uauth if uauth else (username, password)
    response = getattr(requests, verb)(host + endpoint, auth=auth_tup, json=body)
    print('{} {}'.format(response.status_code, response.reason))
    data = response.json()
    if isinstance(data, dict):
        messages = data.pop("messages", None)
        if messages:
            print((json.dumps(messages, indent=4)))
    try:
        response.raise_for_status()
    except Exception as e:
        print(e)
        return None
    else:
        return data

# Check that the user is available
online = espa_api('user')
print((json.dumps(online, indent=4)))

200 OK
{
    "email": "sobczynski@uchicago.edu",
    "first_name": "Alexandra",
    "last_name": "Sobczynski",
    "roles": [
        "active"
    ],
    "username": "sobczynski"
}


In [63]:
##################################
## SUMMARIZE AVAILABLE PRODUCTS ##
##################################

scene_list = three_df_2['scene_id'].tolist()
avail_list = {'inputs': scene_list}

resp = espa_api('available-products', body=avail_list)
#print(json.dumps(resp, indent=4))

200 OK
{
    "tm5_collection_2_l2": {
        "products": [
            "l1",
            "sr_ndvi",
            "sr_evi",
            "sr_savi",
            "sr_msavi",
            "sr_ndmi",
            "sr_nbr",
            "sr_nbr2",
            "sr_ndsi",
            "et"
        ],
        "inputs": [
            "LT05_L2SP_170064_19900623_20200915_02_T1",
            "LT05_L2SP_170064_19930412_20200914_02_T1",
            "LT05_L2SP_170064_19930327_20200914_02_T1",
            "LT05_L2SP_170064_19950402_20200912_02_T1",
            "LT05_L2SP_170064_19950317_20200912_02_T1",
            "LT05_L2SP_170064_19950213_20200912_02_T1",
            "LT05_L2SP_170064_19990429_20200908_02_T1",
            "LT05_L2SP_170064_19990224_20200908_02_T1",
            "LT05_L2SP_170064_19990208_20200908_02_T1",
            "LT05_L2SP_170064_20010504_20200906_02_T1",
            "LT05_L2SP_170064_20010213_20200906_02_T1",
            "LT05_L2SP_170064_20040613_20200903_02_T1",
            "LT05_L

In [75]:
#########################
##### SPECIFY ORDER #####
#########################

order = espa_api('available-products', body=dict(inputs=scene_list))
# I am not deviating from the utm projection, but could
for sensor in order.keys():
    order[sensor]['products'] = ['sr_evi']
order['format'] = 'gtiff'
order['resampling_method'] = 'nn' # nearest neighbor. Other options are cc (cubic convolution) and bil (bilinear interpolation)
order['note'] = 'test download tanzania'
print((json.dumps(order, indent=4)))

200 OK
{
    "tm5_collection_2_l2": {
        "products": [
            "sr_evi"
        ],
        "inputs": [
            "LT05_L2SP_170064_19900623_20200915_02_T1",
            "LT05_L2SP_170064_19930412_20200914_02_T1",
            "LT05_L2SP_170064_19930327_20200914_02_T1",
            "LT05_L2SP_170064_19950402_20200912_02_T1",
            "LT05_L2SP_170064_19950317_20200912_02_T1",
            "LT05_L2SP_170064_19950213_20200912_02_T1",
            "LT05_L2SP_170064_19990429_20200908_02_T1",
            "LT05_L2SP_170064_19990224_20200908_02_T1",
            "LT05_L2SP_170064_19990208_20200908_02_T1",
            "LT05_L2SP_170064_20010504_20200906_02_T1",
            "LT05_L2SP_170064_20010213_20200906_02_T1",
            "LT05_L2SP_170064_20040613_20200903_02_T1",
            "LT05_L2SP_170064_20050616_20200902_02_T1",
            "LT05_L2SP_170064_20050208_20200902_02_T1",
            "LT05_L2SP_170064_20070521_20200830_02_T1",
            "LT05_L2SP_170064_20070403_20200830_

In [76]:
#######################
##### PLACE ORDER #####
#######################

from IPython.display import display, Javascript

def confirm_execute():
    display(Javascript('IPython.notebook.execute_cell()'))

confirmation = input(f"Do you want to order scenes for path: {path} and row: {row_name}? (yes/no)")

if confirmation.lower() == "yes":
    confirm_execute()
else:
    print("Code execution aborted.")
    raise SystemExit("User aborted execution")

place = espa_api('order', verb='post', body=order)
print((json.dumps(place, indent=4)))

201 CREATED
{
    "orderid": "espa-sobczynski@uchicago.edu-04162024-161117-200",
    "status": "ordered"
}


In [115]:
########################
##### CHECK STATUS #####
########################

orderid = place['orderid']
check = espa_api('order-status/{}'.format(orderid))
print((json.dumps(check, indent=4)))

done = espa_api('item-status/{0}'.format(orderid), body={'status': 'complete'})
#print((json.dumps(done[orderid], indent=4)))
num_items = len(done[orderid])
total_items = len(three_df_2['scene_id'])
print(f"{num_items} of {total_items} are processed")

200 OK
{
    "orderid": "espa-sobczynski@uchicago.edu-04162024-161117-200",
    "status": "complete"
}
200 OK
120 of 120 are processed


In [119]:
############################
### WRITE DOWNLOAD FILES ###
############################

urls = [item.get('product_dload_url') for item in done[orderid]]
save_list_to_txt(urls, f"D:/{path_row}_raw/{path_row}.txt")
bat_content = f'for /F "tokens=*" %%a in ({path_row}.txt) do curl -O %%a'
with open(f'D:/{path_row}_raw/{path_row}.bat', 'w') as bat_file:
    bat_file.write(bat_content)